In [1]:
import csv
import datetime as dt
import pandas as pd
import numpy as np

# Data Viz 
import seaborn as sns
import matplotlib.pyplot as plt

# Data Manipulation
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Similarity calculation
from sklearn.metrics.pairwise import cosine_similarity

# Import ML libraries
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds



In [2]:
# Supongamos que 'data' es el DataFrame cargado con las columnas descritas
data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')
data_unida=pd.concat([data,data1,data2]) # Ajusta el nombre del archivo según corresponda
data_unida.head()

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1550/848381883.py:2: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1550/848381883.py:3: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1550/848381883.py:4: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')


,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
0,5514,2016-04-28,ES,H,44,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,5541,2016-04-28,ES,H,60,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2,5631,2016-04-28,ES,V,53,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
3,5656,2016-04-28,ES,H,47,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
4,5738,2016-04-28,ES,V,48,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [3]:
inds=[f'ind_prod{i}' for i in range(1,26)]

In [4]:
data_unida[inds] = data_unida[inds].astype(bool)

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
frequent_itemsets = apriori(data_unida[inds], min_support=0.01, use_colnames=True)

In [6]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Display the association rules
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(ind_prod8),(ind_prod3),0.131909,0.660945,0.087759,0.665301,1.006590,0.000575,1.013014,0.007542
1,(ind_prod3),(ind_prod8),0.660945,0.131909,0.087759,0.132778,1.006590,0.000575,1.001002,0.019310
2,(ind_prod12),(ind_prod3),0.043438,0.660945,0.030423,0.700388,1.059677,0.001713,1.131647,0.058873
3,(ind_prod3),(ind_prod12),0.660945,0.043438,0.030423,0.046030,1.059677,0.001713,1.002717,0.166097
4,(ind_prod14),(ind_prod3),0.018698,0.660945,0.012438,0.665206,1.006447,0.000080,1.012727,0.006527
...,...,...,...,...,...,...,...,...,...,...
1517,(ind_prod23),"(ind_prod25, ind_prod24, ind_prod22, ind_prod5...",0.060312,0.016901,0.016901,0.280232,16.580423,0.015882,1.365855,1.000000
1518,(ind_prod24),"(ind_prod25, ind_prod23, ind_prod22, ind_prod5...",0.128827,0.018687,0.016901,0.131195,7.020817,0.014494,1.129498,0.984381
1519,(ind_prod22),"(ind_prod25, ind_prod23, ind_prod24, ind_prod5...",0.055570,0.018212,0.016901,0.304148,16.700176,0.015889,1.410915,0.995436
1520,(ind_prod5),"(ind_prod25, ind_prod23, ind_prod24, ind_prod2...",0.081740,0.017699,0.016901,0.206770,11.682804,0.015455,1.238356,0.995801


In [7]:
data_unida.loc[data_unida['edad']==' NA','edad']=f"{data_unida['edad'].str.replace(' NA','0').median()}"
data_unida.loc[data_unida['num_antiguedad']=='     NA','num_antiguedad']=f"{data_unida['num_antiguedad'].str.replace('     NA','0').median()}"

In [8]:
data=data_unida[~data_unida['fecha1'].isna()]
data['edad']=data['edad'].astype(float)
data['num_antiguedad']=data['num_antiguedad'].astype(float)
data['fecha1'] = pd.to_datetime(data['fecha1'])
data['mes'] = pd.to_datetime(data['mes'])

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1550/3402631540.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['edad']=data['edad'].astype(float)
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1550/3402631540.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['num_antiguedad']=data['num_antiguedad'].astype(float)
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_1550/3402631540.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [9]:
train_dummy = data.select_dtypes(include = 'O')
train_dummy = pd.get_dummies(train_dummy)
X=train_dummy

In [10]:
pred = {}
for i in inds:
    
    # Use the target product column as the target variable
    if data[i].nunique() == 2:
        
        y = data[i] 
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
    
        # Train the GBDT model
        gbdt_model = GradientBoostingClassifier(n_estimators=300,  # Number of boosting stages
                                           learning_rate=0.1,  # Learning rate
                                           max_depth=3,  # Maximum depth of each tree
                                           random_state=123) 
        gbdt_model.fit(X_train, y_train)
        # Generate GBDT features
        gbdt_features = gbdt_model.apply(X_train)[:, :, 0]
    
        # Train the LR model using the GBDT generated features
        lr_model = LogisticRegression(solver='lbfgs', C=1.0, random_state=123)

        lr_model.fit(gbdt_features, y_train)
    
        # Generate LR features
        gbdt_features_test = gbdt_model.apply(X_test)[:, :, 0]
        lr_features = lr_model.predict_proba(gbdt_features_test)[:, 1]
    
        # Store the predicted score for the product
        pred[i] = lr_features
        
    else: 
        pass
    

/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(
/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer 

KeyboardInterrupt: 

In [ ]:
pred = pd.DataFrame(pred)
test_gbdt = pd.concat([X_test.reset_index(names = 'cod_persona'), pred],axis = 1, ignore_index = True)

col_list = ['cod_persona'] + list(X_test.columns) + list(pred.columns)
test_gbdt.columns = col_list
test_gbdt['cod_persona'] = test_gbdt['cod_persona'].astype('object')

In [ ]:
test_gbdt[list(pred.columns)].sample(10)

,ind_prod1,ind_prod3,ind_prod4,ind_prod5,ind_prod6,ind_prod7,ind_prod8,ind_prod9,ind_prod10,ind_prod11,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
86466,0.000006,0.860458,2.000951e-13,0.001358,5.161358e-08,0.000161,1.667442e-08,2.912892e-09,0.000002,7.538930e-08,...,1.904755e-07,0.000014,0.001201,0.000010,0.000002,7.687415e-14,0.000504,0.000778,0.003001,0.000012
73365,0.000151,0.366057,7.142495e-12,0.002320,9.009478e-06,0.006367,5.788504e-02,1.748872e-02,0.000026,1.833737e-06,...,1.024248e-04,0.000373,0.050684,0.001055,0.000024,2.588881e-03,0.000932,0.001426,0.005181,0.000758
21207,0.000002,0.662489,3.497493e-03,0.260821,8.415187e-06,0.044133,2.524754e-01,1.268099e-01,0.002844,1.986978e-02,...,6.667724e-02,0.000675,0.174465,0.204871,0.236651,9.471727e-03,0.181751,0.216862,0.377196,0.335571
93807,0.000002,0.896160,1.071665e-06,0.078632,1.376585e-07,0.000354,1.203097e-07,1.541232e-08,0.000090,3.212945e-06,...,8.865389e-06,0.000675,0.011322,0.006860,0.001785,4.005483e-14,0.044988,0.044098,0.151164,0.009193
48285,0.000006,0.850251,1.339145e-13,0.001400,8.144947e-08,0.000182,1.535230e-08,2.912892e-09,0.000002,7.538930e-08,...,1.909322e-07,0.000014,0.004141,0.000011,0.000002,3.851401e-14,0.000604,0.000717,0.003514,0.000010
125305,0.000014,0.709427,1.926340e-03,0.172217,3.466013e-02,0.023083,2.296479e-01,8.268969e-02,0.002951,3.942750e-03,...,2.143078e-02,0.002750,0.118701,0.107853,0.052832,5.641094e-03,0.121113,0.129586,0.300861,0.160262
65848,0.000006,0.860458,2.000951e-13,0.001358,5.161358e-08,0.000161,1.667442e-08,2.912892e-09,0.000002,7.538930e-08,...,1.904755e-07,0.000014,0.001201,0.000010,0.000002,7.687415e-14,0.000504,0.000778,0.003001,0.000012
93129,0.000151,0.366057,7.142495e-12,0.002320,9.009478e-06,0.006367,5.788504e-02,1.748872e-02,0.000026,1.833737e-06,...,1.024248e-04,0.000373,0.050684,0.001055,0.000024,2.588881e-03,0.000932,0.001426,0.005181,0.000758
88807,0.000019,0.719794,1.116947e-06,0.222643,1.238510e-04,0.012726,1.035671e-01,4.621982e-02,0.000219,1.642187e-05,...,1.111746e-02,0.001146,0.077085,0.095228,0.016989,1.683617e-07,0.158837,0.223802,0.402286,0.116910
84402,0.000014,0.678763,2.300784e-04,0.106120,3.849793e-05,0.049933,1.114182e-03,2.857292e-05,0.027544,2.605978e-04,...,2.371912e-07,0.002750,0.017321,0.006308,0.004537,8.015374e-04,0.053569,0.072673,0.179578,0.011020


In [42]:
def gbdt_product_recommender(df, cust_id, top_n):
    
    cust_id = str(cust_id)
    prod_list=df.loc[df.index==cust_id,pred.columns].reset_index().T
    prod_list.columns=['valor']
    prod_list = prod_list.sort_values(by = 'valor', ascending = False)
    
    # Ouput the top N recommended products based on the customer's features. If the probability is lower than 0.5 do not output (the customer would not want this one)
    prod_list = prod_list[prod_list['pred_score'] >= 0.001]
    recommend_list = prod_list[0:top_n]
    
    while len(recommend_list) == 0:
        print("Based on the customer's info, there is no bank product recommended for now")
        break
    return recommend_list

In [43]:
gbdt_product_recommender(df = test_gbdt, cust_id = "73365", top_n = 10)

ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [19]:
test_gbdt.to_csv('Predicciones.csv',index=False)

In [45]:
prod_list = test_gbdt.loc[test_gbdt.index==125305,pred.columns].T
prod_list

,125305
ind_prod1,0.000014
ind_prod3,0.709427
ind_prod4,0.001926
ind_prod5,0.172217
ind_prod6,0.034660
ind_prod7,0.023083
ind_prod8,0.229648
ind_prod9,0.082690
ind_prod10,0.002951
ind_prod11,0.003943
